#### Initial Setup

Clone repo, install dependencies, %cd into ./yolov5 folder and check GPU.

In [13]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# !pip install -r yolov5/requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 15679, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15679 (delta 1), reused 2 (delta 0), pack-reused 15672
Receiving objects: 100% (15679/15679), 14.45 MiB | 34.57 MiB/s, done.
Resolving deltas: 100% (10742/10742), done.
/nfs/home/zle2435/dl_proj/notebook/yolov5


In [1]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.0.1+cu117 _CudaDeviceProperties(name='NVIDIA RTX A6000', major=8, minor=6, total_memory=48685MB, multi_processor_count=84)


#### Set up folder structure before training - Done

remove the class:1 circle; remove these

In [6]:
# check the .txt files for class
import os

folder_path = '../data/annotated'
delete_list = []

# Walk through the directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            # Open the file and check the first letter
            with open(file_path, 'r') as f:
                first_line = f.readline().strip()
                if first_line.startswith('1'):
                    file_name_without_extension = os.path.splitext(file)[0]
                    delete_list.append(file_name_without_extension)
                                        

In [7]:
delete_list

[]

In [11]:
# delete all files in the folder that has the substring form the delete_list
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        
        # Check if any substring from delete_list is present in the file name
        if any(substring in file for substring in delete_list):
            try:
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
            except OSError as e:
                print(f"Error deleting file: {file_path}\nError message: {e}")

Deleted file: ../data/annotated/image_349.txt
Deleted file: ../data/annotated/image_244.jpg
Deleted file: ../data/annotated/image_148.txt
Deleted file: ../data/annotated/image_148.jpg
Deleted file: ../data/annotated/image_349.jpg
Deleted file: ../data/annotated/image_244.txt


Update data folder to get the right folder structure for training: train and val. 60/15/25 split for train/val/test. Since the annoted images are already randomly sampled - we just break it down to train/val/test sequentially.

In [11]:
%mkdir ../data/transfer_learning

In [12]:
%mkdir ../data/transfer_learning/train/
%mkdir ../data/transfer_learning/val/
%mkdir ../data/transfer_learning/test/

In [13]:
import shutil, os
# get 240 images for training and 60 images for validation
file_path = '../data/annotated'
train_path = '../data/transfer_learning/train/'
val_path = '../data/transfer_learning/val/'
test_path = '../data/transfer_learning/test/'

counter = 0
for root, dirs, files in os.walk(file_path):
    for file in files:
        if file.endswith('.jpg'):
            file_name_without_extension = os.path.splitext(file)[0]
            file_txt = file_name_without_extension + '.txt'
            if counter < 240:
                file_path = os.path.join(root, file)
                file_txt_path = os.path.join(root, file_txt)
                shutil.copy(file_path, train_path)
                shutil.copy(file_txt_path, train_path)
            elif counter < 240 + 60:
                file_path = os.path.join(root, file)
                file_txt_path = os.path.join(root, file_txt)
                shutil.copy(file_path, val_path)
                shutil.copy(file_txt_path, val_path)
            else: # test should have 57 images, since we dropped the 3 circles
                file_path = os.path.join(root, file)
                file_txt_path = os.path.join(root, file_txt)
                shutil.copy(file_path, test_path)
                shutil.copy(file_txt_path, test_path)
            counter += 1

In [14]:
# just in case, we re-size all images to 640 pixels
from PIL import Image
import os

# Set the path to the directory containing your images
image_dir = '../data/transfer_learning/'

# Set the target width for resizing
target_width = 640

# Iterate over each image in the directory
for root, dirs, files in os.walk(image_dir):
     for filename in files:
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Open the image file
            image_path = os.path.join(root, filename)
            img = Image.open(image_path)
            
            # Calculate the proportional height based on the target width
            width, height = img.size
            target_height = int((height / width) * target_width)
            
            # Resize the image using the target width and proportional height
            resized_img = img.resize((target_width, target_height))
            
            # Save the resized image, overwriting the original file
            resized_img.save(image_path)

            print(f"Resized '{filename}' to {target_width} pixels width")

Resized 'image_352.jpg' to 640 pixels width
Resized 'image_205.jpg' to 640 pixels width
Resized 'image_363.jpg' to 640 pixels width
Resized 'image_183.jpg' to 640 pixels width
Resized 'image_307.jpg' to 640 pixels width
Resized 'image_139.jpg' to 640 pixels width
Resized 'image_265.jpg' to 640 pixels width
Resized 'image_123.jpg' to 640 pixels width
Resized 'image_166.jpg' to 640 pixels width
Resized 'image_152.jpg' to 640 pixels width
Resized 'image_197.jpg' to 640 pixels width
Resized 'image_85.jpg' to 640 pixels width
Resized 'image_92.jpg' to 640 pixels width
Resized 'image_154.jpg' to 640 pixels width
Resized 'image_335.jpg' to 640 pixels width
Resized 'image_395.jpg' to 640 pixels width
Resized 'image_302.jpg' to 640 pixels width
Resized 'image_330.jpg' to 640 pixels width
Resized 'image_333.jpg' to 640 pixels width
Resized 'image_204.jpg' to 640 pixels width
Resized 'image_285.jpg' to 640 pixels width
Resized 'image_326.jpg' to 640 pixels width
Resized 'image_141.jpg' to 640 pix

In [67]:
%ls -1 ../data/annotated/ | wc -l

794


In [16]:
prefix = ['train', 'val', 'test']
total_count = 794/2

After a tunning - we used only 1 image for validation placeholder. We want more training images because we only have 400 images. Also note that yolo5s-CXR.yaml actually uses yolo5x - same architecture. 

In [70]:
for p in prefix:
    all_images = []
    image_dir = f'../data/transfer_learning/{p}/'
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'): 
            all_images.append(filename)
    print(f"% of images in {p}: {len(all_images)/total_count}")

% of images in train: 0.7531486146095718
% of images in val: 0.0025188916876574307
% of images in test: 0.24433249370277077


#### Set up reference before training

Done in an editor:

- create new data yaml files (train, val reference) -> data/CXR.yaml
- update the yolov5x.ymal regarding the class numbers  -> models/yolov5s-CXR.yaml
- duplicate yolov5n.pt and yolov5s.pt to the /weights folder -> weights/yolov5s-CXR.pt

In [18]:
%cd ../yolov5/

/nfs/home/zle2435/dl_proj/yolov5


#### Transfer Learning 
with only architecture (not freezing any layers) and no agumentation (based on the yaml file) - the full architecture can be found at [ultralytics](https://docs.ultralytics.com/yolov5/tutorials/architecture_description/)

In [19]:
# train yolov5s on custom data for 1 epochs
!python3 train.py --weights '' --hyp data/hyps/hyp.no-augmentation.yaml --cfg ./models/yolov5x-CXR.yaml --img 640 --epochs 32 --batch 32 --data ./data/CXR.yaml --device 0,1,2,3

train: weights=, cfg=./models/yolov5s-CXR.yaml, data=./data/CXR.yaml, hyp=data/hyps/hyp.no-augmentation.yaml, epochs=32, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0,1,2,3, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 230 (delta 165), reused 176 (delta 154), pack-reused 36
Receiving objects: 100% (230/230), 75.77 KiB | 5.41 MiB/s, done.
Resolving deltas: 100% (183/183), completed with 25 local objects.
From https://github.com/ultralytics/yolov5
 

#### Inference 
with test data and check result

In [22]:
!python3 detect.py --source data/transfer_learning/test/ --weights runs/train/exp8/weights/best.pt --img 640 --save-txt --device 0,1,2,3 --conf-thres 0.0001 --iou-thres 0.0001 --max-det 1 --hide-conf

detect: weights=['runs/train/exp8/weights/best.pt'], source=data/transfer_learning/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.0001, iou_thres=0.0001, max_det=1, device=0,1,2,3, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=True, half=False, dnn=False, vid_stride=1
requirements: /nfs/home/zle2435/dl_proj/.venv/lib/python3.10/site-packages/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-168-gec2b853 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48685MiB)
                                                            CUDA:2 (NVIDIA RTX A6000, 48685MiB)
                                                            CUDA:3 (NVIDIA RTX A6000, 48685MiB)

Fusing l

The results can now be found at the `runs/detect/{exp_index}`. Let's test a pair of box coordinates (x, y, w, h) and calculate the iou from the test folder - we are checking prediction against groud truth of course. 

In [28]:
from utils.metrics import bbox_iou

In [29]:
%pwd

'/nfs/home/zle2435/dl_proj/yolov5'

In [47]:
def get_coord(dir:str, file_name:str) -> torch.Tensor:
    """from a .txt file, read the first line, and get the (x,y,w,h) coordinates"""
    with open(dir + file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        box_coords = [float(coord) for coord in lines[0].split()[1:]] # first elem is class
    return torch.Tensor(box_coords)

In [48]:
ground_truth_dir = r'../data/transfer_learning/test/'
prediction_dir = r'runs/detect/exp28/labels/'
file_name = 'image_4.txt'

box1 = get_coord(ground_truth_dir, file_name)
box2 = get_coord(prediction_dir, file_name)
iou = bbox_iou(box1, box2)

print("Box 1:", box1)
print("Box 2:", box2)
print("IoU:", iou)

Box 1: tensor([0.2218, 0.1087, 0.0355, 0.0378])
Box 2: tensor([0.9812, 0.0701, 0.0375, 0.1041])
IoU: tensor([0.])


In [65]:
import os

ground_truth_dir = r'../data/transfer_learning/test/'
prediction_dir = r'runs/detect/exp28/labels/'
iou_list = []

# iterate the ground truth directory
for root, dirs, files in os.walk(ground_truth_dir):
    # check if the file is a .txt file
    for file in files:
        if '.txt' not in file: 
            continue
        else:
            file_prefix = file.split('.')[0]
        box1 = get_coord(ground_truth_dir, file)
        box2 = get_coord(prediction_dir, file)
        iou = float(bbox_iou(box1, box2))
        print(f'for {file_prefix}, groud truth:{box1}, pred:{box2}, IoU is {iou}')
        iou_list.append(iou)

for image_107, groud truth:tensor([0.4886, 0.8776, 0.5742, 0.1491]), pred:tensor([0.8375, 0.7684, 0.0688, 0.0970]), IoU is 0.0
for image_108, groud truth:tensor([0.4948, 0.5658, 0.7181, 0.3464]), pred:tensor([0.9516, 0.0493, 0.0969, 0.0986]), IoU is 0.0
for image_144, groud truth:tensor([0.5184, 0.1011, 0.4176, 0.1904]), pred:tensor([0.1594, 0.7943, 0.3187, 0.4114]), IoU is 0.0
for image_327, groud truth:tensor([0.3374, 0.0788, 0.3499, 0.1159]), pred:tensor([0.3039, 0.1179, 0.0734, 0.1066]), IoU is 0.1229923814535141
for image_126, groud truth:tensor([0.4378, 0.5866, 0.1061, 0.0918]), pred:tensor([0.9500, 0.9476, 0.1000, 0.1049]), IoU is 0.0
for image_162, groud truth:tensor([0.4884, 0.0718, 0.0824, 0.1260]), pred:tensor([0.8773, 0.0442, 0.0453, 0.0885]), IoU is 0.0
for image_151, groud truth:tensor([0.7812, 0.6193, 0.0658, 0.1292]), pred:tensor([0.8797, 0.9491, 0.1031, 0.1019]), IoU is 0.0
for image_64, groud truth:tensor([0.6377, 0.5913, 0.2643, 0.1631]), pred:tensor([0.0477, 0.7457,

In [66]:
# calculate the average of the iou_list
sum(iou_list)/len(iou_list)

0.006351515828372584